<a href="https://colab.research.google.com/github/luciainnocenti/IncrementalLearning/blob/Classifier--KNN/ICaRLMain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import GitHub repository

In [ ]:
import os
import logging
import sys

In [ ]:
if not os.path.isdir('./DatasetCIFAR'):
  !git clone -b Classifier--KNN https://github.com/luciainnocenti/IncrementalLearning.git
  !mv 'IncrementalLearning' 'DatasetCIFAR'

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (211/211), done.
remote: Total 2419 (delta 138), reused 5 (delta 5), pack-reused 2203
Receiving objects: 100% (2419/2419), 2.87 MiB | 5.34 MiB/s, done.
Resolving deltas: 100% (1548/1548), done.


# Import packages

In [ ]:
from DatasetCIFAR.data_set import Dataset 
from DatasetCIFAR.data_set import Subset
from DatasetCIFAR import ResNet
from DatasetCIFAR import utils
from DatasetCIFAR import params
from DatasetCIFAR import ICaRLModel
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

from torchvision import transforms
from torch.utils.data import DataLoader
from torch.nn import functional as F
import random
random.seed(params.SEED)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
print(params.SEED)
print(params.NUM_WORKERS)

653
4


# Define Datasets

In [ ]:
train_transformer = transforms.Compose([transforms.RandomCrop(size = 32, padding=4),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transformer = transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
trainDS = Dataset(train=True)
testDS = Dataset(train=False)

Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified


In [ ]:
train_splits = trainDS.splits
test_splits = testDS.splits

# Define Network

In [ ]:
ICaRL = ResNet.resnet32(num_classes=100)
ICaRL =  ICaRL.to(params.DEVICE)

In [ ]:
exemplars = [None]*100

test_indexes =  []
accs = []

In [ ]:
for task in range(0, 100, params.TASK_SIZE):
  train_indexes = trainDS.__getIndexesGroups__(task)
  test_indexes = test_indexes + testDS.__getIndexesGroups__(task)

  train_dataset = Subset(trainDS, train_indexes, transform = train_transformer)
  test_dataset = Subset(testDS, test_indexes, transform = test_transformer)

  train_loader = DataLoader( train_dataset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE, shuffle=True)
  test_loader = DataLoader( test_dataset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE , shuffle=True )
  
  ICaRL, exemplars = ICaRLModel.incrementalTrain(task, trainDS, ICaRL, exemplars, train_transformer)

  col = []
  for i,x in enumerate( train_splits[ :int(task/10) + 1]) : 
    v = np.array(x)
    col = np.concatenate( (col,v), axis = None)
    col = col.astype(int)
  
  total = 0.0
  running_corrects = 0.0
  for img, lbl, _ in train_loader:
      img = img.float().to(params.DEVICE)
      preds = ICaRLModel.classify(img, exemplars, ICaRL, task, trainDS)
      preds = preds.to(params.DEVICE)

      total += len(lbl)
      running_corrects += torch.sum(preds == lbl.data.to(params.DEVICE)).data.item()

  accuracy = float(running_corrects/total)
  print(f'task: {task}', f'train accuracy = {accuracy}')
  accs.append(accuracy)

  total = 0.0
  running_corrects = 0.0

  for img, lbl, _ in test_loader:
      img = img.float().to(params.DEVICE)
      preds = ICaRLModel.classify(img, exemplars, ICaRL, task, trainDS)
      preds = preds.to(params.DEVICE)


      total += len(lbl)
      running_corrects += torch.sum(preds == lbl.data.to(params.DEVICE)).data.item()

  accuracy = float(running_corrects/total)
  print(f'task: {task}', f'test accuracy = {accuracy}')

In [ ]:
X_train = []

In [ ]:
X_train.append([3])
X_train.append([5])

In [ ]:
X_train

In [ ]:
accs = [.897,.80,.76,.69,.64,.58,.54,.52,.48,.46]

In [ ]:
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
def plotTask(pars_tasks):

  x_tasks =  np.linspace(10, 100, 10)

  plt.plot(x_tasks, pars_tasks, label=['Accuracy'])
  plt.xlabel("Epoch")
  plt.ylim(0,1)
  plt.title('Accuracy over tasks')
  plt.legend(['Accuracy'])
  plt.grid(True)
  plt.show()

In [ ]:
plotTask(accs)

In [ ]:
np.diagonal(cf)

In [ ]:
true = np.sum(cf, axis=1)
true

In [ ]:
pred = np.sum(cf, axis=0)
pred

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pred = np.array([137,  98, 142, 112,  39,  82, 116,  96, 145, 112,  84, 110,  59,
        96,  73,  78,  50,  91,  98,  92,  64, 109,  91,  72,  38,  70,
        76,  81,  89, 102,  81, 116,  72,  59, 177,  59,  62,  63,  93,
        90,  72,  79, 102,  52, 162,  68,  67,  80,  65,  98,  80,  65,
        84,  96, 121,  53,  77, 136,  69, 103,  76,  94, 109,  79, 126,
        61, 126, 106, 182,  68, 122,  94,  90,  80,  80, 159,  98, 132,
       145,  88, 170,  77,  79,  78, 131,  74, 149,  70, 149, 114, 102,
       172, 212, 134, 124, 187, 189, 114, 126, 131])

In [ ]:
val = np.arange(100)

In [ ]:
avg = np.mean(pred)
avg

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)
x_coordinates = val
ax.bar(x_coordinates, pred, align='center')
ax.axhline(avg, color="gray")
plt.show()

In [ ]:
i = 0
for el, name in zip(pred,trainDS.__getClassesNames__()) :
  print('classe', i, 'name', name, ',preds', el)
  i+=1

In [ ]:
train_splits

In [ ]:
trainDS.__getClassesNames__()